# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carnarvon,-24.8667,113.6333,21.52,62,clear sky,8.90,AU,1697741107
1,1,hamilton,39.1834,-84.5333,17.07,77,overcast clouds,4.63,US,1697740934
2,2,bethel,41.3712,-73.4140,19.00,58,few clouds,3.60,US,1697741107
3,3,minas de marcona,-15.2119,-75.1103,21.79,56,clear sky,6.10,PE,1697741107
4,4,blackmans bay,-43.0167,147.3167,9.18,93,overcast clouds,1.20,AU,1697741048


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [20]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 700,
    size = "Humidity",
    scale = 1,
    color = "City"

)

# Display the map plot_1
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [25]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[city_data_df['Humidity']<=50,:]

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
5,5,puerto natales,-51.7236,-72.4875,15.25,36,scattered clouds,8.75,CL,1697741107
22,22,samara,53.0000,50.0000,4.53,45,overcast clouds,3.97,RU,1697741110
34,34,lake city,30.1897,-82.6393,26.03,43,clear sky,1.54,US,1697741111
43,43,qaqortoq,60.7167,-46.0333,-1.29,46,clear sky,1.39,GL,1697741114
47,47,gaiman,-43.2897,-65.4929,27.11,17,scattered clouds,2.84,AR,1697740940
...,...,...,...,...,...,...,...,...,...,...
559,559,fort bragg,35.1390,-79.0060,23.69,44,clear sky,2.57,US,1697741096
560,560,ariquemes,-9.9133,-63.0408,42.70,15,scattered clouds,1.70,BR,1697741011
563,563,sikandra,26.3667,79.6500,23.01,43,clear sky,2.17,IN,1697741179
566,566,arkalyk,50.2486,66.9114,5.74,42,overcast clouds,4.89,KZ,1697741179


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
5,puerto natales,CL,-51.7236,-72.4875,36,
22,samara,RU,53.0000,50.0000,45,
34,lake city,US,30.1897,-82.6393,43,
43,qaqortoq,GL,60.7167,-46.0333,46,
47,gaiman,AR,-43.2897,-65.4929,17,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [55]:
# Set parameters to search for a hotel
radius = 10000

params = {
    'categories':'accommodation.hotel',
    'filter': "",
    'bias': "",
    'apiKey': geoapify_key,
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,'Lat']
    longitude = hotel_df.loc[index,'Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params) 
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
     # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
       hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
     # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
puerto natales - nearest hotel: Dorotea Patagonia Hostel
samara - nearest hotel: No hotel found
lake city - nearest hotel: Cypress Inn
qaqortoq - nearest hotel: Hotel Qaqortoq
gaiman - nearest hotel: yr hen ffordd
semera - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
turpan - nearest hotel: Oriental
slave lake - nearest hotel: Northwest Inn
sicuani - nearest hotel: Hostal El Imperador
tabora - nearest hotel: Alpho hotel
mandalgovi - nearest hotel: АЛТАНГОВЬ Hotel
tres arroyos - nearest hotel: Andrea Hotel
punta arenas - nearest hotel: Hotel Lacolet
santa cruz - nearest hotel: Aviador Hotel Boutique
'ain el hadjar - nearest hotel: Hôtel Forsane
azadshahr - nearest hotel: هتل آموزش
crystal city - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
aioun - nearest hotel: فندق العيون
sa'dah - nearest hotel: الشارقة مول للمواد الغذائية
winnemucca - nearest hotel: The Overland Hotel
paragominas - neares

,City,Country,Lat,Lng,Humidity,Hotel Name
5,puerto natales,CL,-51.7236,-72.4875,36,Dorotea Patagonia Hostel
22,samara,RU,53.0000,50.0000,45,No hotel found
34,lake city,US,30.1897,-82.6393,43,Cypress Inn
43,qaqortoq,GL,60.7167,-46.0333,46,Hotel Qaqortoq
47,gaiman,AR,-43.2897,-65.4929,17,yr hen ffordd
...,...,...,...,...,...,...
559,fort bragg,US,35.1390,-79.0060,44,Airborne Inn Lodging
560,ariquemes,BR,-9.9133,-63.0408,15,Aquarius
563,sikandra,IN,26.3667,79.6500,43,No hotel found
566,arkalyk,KZ,50.2486,66.9114,42,"Гостиница ""Арсенал"""


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
# Configure the map plot
map_plotz = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 700,
    size = "Hotel",
    scale = 1,
    color = "City"

)

# Display the map plot_1
map_plotz

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)